# Lab 3 – Module 0: When Straight Lines Fail

**Time:** ~5 minutes

---

In Labs 1 and 2 you used straight lines to fit data and learned how gradient descent finds the best slope and intercept.

**Today’s question:** What if NO straight line works at all?

Below you’ll see three dot patterns. Your job is simple: **drag the sliders to draw a line that puts all the blue dots on one side and all the red dots on the other.**

For one pattern this is easy. For the other two… you’ll see.

## 1. Create Three Datasets

Run this cell to generate three classification problems. You don’t need to change anything — just press the play button.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import FloatSlider, interact
from IPython.display import display

np.random.seed(42)

# Dataset 1: Two clouds (separable)
n = 100
X1_c0 = np.random.randn(n // 2, 2) * 0.5 + np.array([-1.5, -1.5])
X1_c1 = np.random.randn(n // 2, 2) * 0.5 + np.array([1.5, 1.5])
X1 = np.vstack([X1_c0, X1_c1])
y1 = np.hstack([np.zeros(n // 2), np.ones(n // 2)])

# Dataset 2: XOR pattern (four corners, diagonal classes)
n_corner = 25
corners = [(-1.5, -1.5), (1.5, -1.5), (-1.5, 1.5), (1.5, 1.5)]
labels  = [0, 1, 1, 0]  # diagonal pairs share a class
X2, y2 = [], []
for (cx, cy), lab in zip(corners, labels):
    pts = np.random.randn(n_corner, 2) * 0.3 + np.array([cx, cy])
    X2.append(pts)
    y2.append(np.full(n_corner, lab))
X2 = np.vstack(X2)
y2 = np.hstack(y2)

# Dataset 3: Ring (inner dot vs outer ring)
n_ring = 50
ang_in  = np.random.uniform(0, 2 * np.pi, n_ring)
r_in    = np.random.uniform(0.3, 0.8, n_ring)
X3_in   = np.column_stack([r_in * np.cos(ang_in), r_in * np.sin(ang_in)])
ang_out = np.random.uniform(0, 2 * np.pi, n_ring)
r_out   = np.random.uniform(1.5, 2.2, n_ring)
X3_out  = np.column_stack([r_out * np.cos(ang_out), r_out * np.sin(ang_out)])
X3 = np.vstack([X3_in, X3_out])
y3 = np.hstack([np.zeros(n_ring), np.ones(n_ring)])

print('Three datasets ready!')
print('  Dataset 1 – Two Clouds')
print('  Dataset 2 – XOR (four corners)')
print('  Dataset 3 – Ring (inner dot vs outer ring)')

## 2. See All Three Patterns

Take a quick look. Blue and red are two different classes.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4), dpi=100)
for ax, X, y, title in zip(axes,
                            [X1, X2, X3],
                            [y1, y2, y3],
                            ['Dataset 1: Two Clouds',
                             'Dataset 2: XOR Pattern',
                             'Dataset 3: Ring']):
    ax.scatter(X[y == 0, 0], X[y == 0, 1], c='blue',  s=50, alpha=0.6, edgecolors='k', label='Class 0')
    ax.scatter(X[y == 1, 0], X[y == 1, 1], c='red',   s=50, alpha=0.6, edgecolors='k', label='Class 1')
    ax.set_xlabel('x\u2081'); ax.set_ylabel('x\u2082')
    ax.set_title(title, fontweight='bold')
    ax.legend(fontsize=9); ax.grid(True, alpha=0.3); ax.set_aspect('equal')
plt.tight_layout()
plt.show()

## 3. Try to Separate Each Pattern with a Straight Line

Use the **Dataset** slider to switch between the three patterns.  
Use **Slope** and **Intercept** to move the green line.  
The accuracy tells you what percentage of dots are on the correct side.

**Goal:** Get as close to 100 % as you can on each dataset.

In [ ]:
def try_line(dataset_num, slope, intercept):
    X, y, title = [(X1, y1, 'Two Clouds'),
                   (X2, y2, 'XOR Pattern'),
                   (X3, y3, 'Ring')][dataset_num - 1]

    fig, ax = plt.subplots(figsize=(7, 7), dpi=100)
    ax.scatter(X[y == 0, 0], X[y == 0, 1], c='blue', s=70, alpha=0.6,
              edgecolors='k', label='Class 0')
    ax.scatter(X[y == 1, 0], X[y == 1, 1], c='red',  s=70, alpha=0.6,
              edgecolors='k', label='Class 1')

    xr = np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 100)
    ax.plot(xr, slope * xr + intercept, 'g-', linewidth=3,
            label=f'Line: x\u2082 = {slope:.1f}\u00b7x\u2081 + {intercept:.1f}')

    pred = (X[:, 1] > slope * X[:, 0] + intercept).astype(int)
    acc  = np.mean(pred == y) * 100

    ax.set_title(f'{title}  \u2014  Accuracy: {acc:.0f}%', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10); ax.grid(True, alpha=0.3); ax.set_aspect('equal')
    ax.set_xlim(X[:, 0].min() - 0.5, X[:, 0].max() + 0.5)
    ax.set_ylim(X[:, 1].min() - 0.5, X[:, 1].max() + 0.5)
    plt.tight_layout(); plt.show()

    if acc >= 95:
        print(f'Nice! {acc:.0f}% \u2014 the line separates the classes well.')
    elif acc >= 60:
        print(f'{acc:.0f}% \u2014 some dots are on the wrong side. Keep adjusting!')
    else:
        print(f'{acc:.0f}% \u2014 roughly coin-flip territory. Maybe a straight line just can\u2019t do it?')

interact(
    try_line,
    dataset_num=widgets.IntSlider(min=1, max=3, step=1, value=1, description='Dataset:'),
    slope=FloatSlider(min=-3, max=3, step=0.1, value=1.0, description='Slope:'),
    intercept=FloatSlider(min=-3, max=3, step=0.1, value=0.0, description='Intercept:')
);

## 4. What You Should Have Found

| Dataset | Can a line separate it? | Why? |
|---------|------------------------|------|
| **Two Clouds** | Yes | The blue cloud is on one side, the red cloud on the other. |
| **XOR** | No | Diagonal corners share a class — any line that gets two corners right puts the other two on the wrong side. |
| **Ring** | No | The blue dots are surrounded by red dots in a circle. A straight line always cuts through both classes. |

**Bottom line:** Some patterns simply cannot be solved by a straight line. We need something smarter — and that’s what the rest of this lab is about.

## Answer‑Sheet Questions (Q1 – Q3)

**Q1.** Which dataset could you separate perfectly with a straight line? Describe what happened when you tried the slider on the other two — what kept going wrong no matter how you adjusted it?

**Q2.** For the **ring** dataset, sketch or describe what a boundary would have to look like to correctly put the inner circle on one side and the outer ring on the other. Why is a straight line doomed before you even start?

**Q3.** Based on what you saw, what is one thing a smarter boundary would need to be able to **do** that a straight line simply cannot?

---

**Next:** Continue to **Module 1** to see how *activation functions* solve this problem by bending space itself.